# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [93]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [94]:
weather_df = pd.read_csv('../WeatherPY/weather_data.csv')
weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bluff,-46.60,168.33,39.00,83,3,3.00,NZ,1592485233
1,1,Belaya Kholunitsa,58.84,50.85,65.62,38,0,6.64,RU,1592485255
2,2,Kapaa,22.08,-159.32,77.00,83,20,17.22,US,1592485202
3,3,Vestmannaeyjar,63.44,-20.27,51.80,71,40,23.04,IS,1592485259
4,4,Vaini,-21.20,-175.20,68.00,94,75,8.05,TO,1592485156


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [96]:
#drop na values
weather_df = weather_df.dropna()
#define location and weight
weight = weather_df["Humidity"].astype(float)
locations = weather_df[['Lat', 'Lng']].astype(float)
#configure maps
gmaps.configure(api_key=g_key)
fig_layout={'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=fig_layout,zoom_level=1,center=(0,0))


heat_layer = gmaps.heatmap_layer(locations, weights=weight,max_intensity=max(weather_df["Humidity"]),point_radius = 2)

figure.add_layer(heat_layer)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [97]:
new_weather_df = weather_df
#A max temperature lower than 80 degrees but higher than 70.
new_weather_df= new_weather_df.loc[(new_weather_df["Max Temp"] < 80) & (new_weather_df["Max Temp"] > 70)]
#Wind speed less than 10 mph.
new_weather_df=new_weather_df.loc[(new_weather_df["Wind Speed"] < 10)]
#Zero cloudiness.
new_weather_df=new_weather_df.loc[(new_weather_df["Cloudiness"] ==0)]

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

#making sure there are less than 10 locations-- found 9 locations
#locations_count=new_weather_df["Lat"].nunique()

new_weather_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,Hermanus,-34.42,19.23,75.00,37,0,1.01,ZA,1592485291
20,20,Bredasdorp,-34.53,20.04,77.00,27,0,6.29,ZA,1592485295
219,219,São João da Barra,-21.64,-41.05,71.60,78,0,4.70,BR,1592485712
268,268,Kieta,-6.22,155.63,79.41,79,0,3.60,PG,1592485818
356,356,Karratha,-20.74,116.85,72.63,58,0,4.85,AU,1592486014
409,409,Kruisfontein,-34.00,24.73,71.38,32,0,9.48,ZA,1592486126
433,433,Pengcheng,36.43,114.17,73.38,64,0,3.36,CN,1592486178
439,439,Vytegra,61.01,36.45,77.38,44,0,5.88,RU,1592486191
448,448,Matamoros,25.53,-103.25,74.50,50,0,1.72,MX,1592486210


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [99]:
#set up parameters
params = {
    "radius": 5000,
    "ke word":"hotel",
    "types": "lodging",
    "key": g_key
}

# find lat lng in hotel df to get location in parameters
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each time but leave the other paramets the same
    params["location"] = f"{lat},{lng}"

    # make API request and get results in json format
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel = requests.get(base_url, params=params)
    hotel=hotel.json()
    #pprint(hotel)

   
    # add hotel name to hotel dataframe.there may not be hotels nearby so need skip when missing value occurs.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    
    except (KeyError, IndexError):
        #print("No nearby hotel... skipping.")
        print("")
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,18,Hermanus,-34.42,19.23,75.00,37,0,1.01,ZA,1592485291,Misty Waves Boutique Hotel
20,20,Bredasdorp,-34.53,20.04,77.00,27,0,6.29,ZA,1592485295,Bredasdorp Country Manor
219,219,São João da Barra,-21.64,-41.05,71.60,78,0,4.70,BR,1592485712,Pousada Porto De Canoas
268,268,Kieta,-6.22,155.63,79.41,79,0,3.60,PG,1592485818,DJ Garden Lodge
356,356,Karratha,-20.74,116.85,72.63,58,0,4.85,AU,1592486014,ibis Styles Karratha
409,409,Kruisfontein,-34.00,24.73,71.38,32,0,9.48,ZA,1592486126,Oyster Bay House Rental
433,433,Pengcheng,36.43,114.17,73.38,64,0,3.36,CN,1592486178,Fengfeng Dianchang Hotel
439,439,Vytegra,61.01,36.45,77.38,44,0,5.88,RU,1592486191,Wardenclyffe Volgo&Balt
448,448,Matamoros,25.53,-103.25,74.50,50,0,1.72,MX,1592486210,Recreativo Sabdy Briseidy


In [100]:

# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [102]:
# Add marker layer ontop of heat map
fig_layout={'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=fig_layout,zoom_level=2,center=(0,0))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…